In [1]:

import sys
sys.setrecursionlimit(99999)
import os
import glob
import cv2
import math
import pickle
import datetime
import pandas as pd
import statistics
import keras
import keras.backend as K
import h5py
import numpy as np
np.random.seed(2016)

from sklearn.cross_validation import KFold

from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss
from keras import optimizers
from keras.optimizers import RMSprop, SGD
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers import Input, merge, Activation, Dropout, Flatten, Dense
from keras.models import Model, model_from_json
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2


Using Theano backend.
Using gpu device 0: GeForce GTX TITAN X (CNMeM is enabled with initial size: 75.0% of memory, cuDNN 5105)
/home/ipl/anaconda3/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [3]:
#現在可以跳過~
train_data_dir = '/home/ipl/Fish/C0_rotate'
def build(source=None):
    datagen = ImageDataGenerator(rescale=1. / 255)
    data_generator = datagen.flow_from_directory(
    source,  # this is the target directory
    target_size=(256, 256),  # all images will be resized to 150x150
    batch_size=11,
    class_mode='categorical')
    class_dictionary = data_generator.class_indices
    return data_generator, class_dictionary

In [4]:
data_generator,class_dictionary=build(train_data_dir)

Found 6740 images belonging to 146 classes.


In [5]:
path=data_generator.directory

In [12]:
indice=data_generator.class_indices
arrindice=list(sorted((indice.keys())))

In [13]:
arrindice

['Alaska Skate',
 'Alcyonidium pedunculatum',
 'Aleutian Alligatorfish',
 'Aleutian Skate',
 'Antlered Sculpin',
 'Argis ovifer',
 'Arrowtooth Flounder',
 'Ascidia paratropa',
 'Atka Mackerel',
 'Bairdi Tanner Crab',
 'Balanus Sp',
 'Barnacle',
 'Bathymaster Signatus',
 'Bathyplotes sp',
 'Bathyraja mariposa',
 'Bathyraja maripose egg case',
 'Berryteuthis magister',
 'Black Rockfish',
 'Black Spotted Rockfish',
 'Brown King Crab',
 'Bryozoichthys lysimus',
 'Bryozoichthys marjorius',
 'Calcigorgia spiculifera',
 'Ceramaster Sp',
 'Cheisraster Dawsinii',
 'Chinook Salmon',
 'Chrysaora melanaster',
 'Chylamys Sp',
 'Clinocardium Sp',
 'Craniella Sp',
 'Crella brunnea',
 'Crossaster papposus',
 'Cryogorgia koolsae',
 'Cucumaria fallax',
 'Dark Rockfish',
 'Decapodiformes',
 'Diplopteraster multipes',
 'Docked Lumpsucker',
 'Docked Snailfish',
 'Dover Sole',
 'Dusky Rockfish',
 'Evasterias',
 'Fanellia compressa',
 'Flathead Sole',
 'Fusitriton oregonensis',
 'Ghost Sponge',
 'Giant Grena

In [6]:
#2016/11/12 04:02  LOAD DATA !!!!

# 1. 用data generater讀取圖片位址跟class
# 現在只要用load就可以了 
X=np.load('/home/ipl/Fish/X.npy')
y=np.load('/home/ipl/Fish/y.npy')
y_binary=np.load('/home/ipl/Fish/y_binary.npy')


#這個版本式沒有做Data Augumetation的
###會不會做normalization有影響? 
#因為現在是有做的，可以試試看沒有做的?


In [9]:
train_data=X
train_data.shape

(6740, 3, 224, 224)

In [10]:

#2. shuffle
from sklearn.utils import shuffle
train_data_shuffle,y_shuffle, y_binary_shuffle = shuffle(train_data,y, y_binary, random_state=0)


In [26]:
#3.#20161113 04:16  train_test_split   抽出後面２０%  
#######################################
X_train, X_test, y_train, y_test = train_test_split(train_data_shuffle, y_binary_shuffle, test_size=0.33, random_state=42)

In [27]:
print("X_train.shape:  ",X_train.shape)
print("y_train.shape:  ",y_train.shape)
print("------")
print("X_test.shape:  ",X_test.shape)
print("y_test.shape:  ",y_test.shape)

X_train.shape:   (4515, 3, 224, 224)
y_train.shape:   (4515, 146)
------
X_test.shape:   (2225, 3, 224, 224)
y_test.shape:   (2225, 146)


In [29]:
#y_train[5]

In [16]:
def get_actual_pred_proba(raw_pred_proba):
    pred_proba = np.zeros((len(raw_pred_proba), 146))
    for i, label in enumerate(np.unique(y_train)):
        print(i,"  Lable: ",label)
        pred_proba[:, label] = raw_pred_proba[:, i]
    return pred_proba

In [ ]:
#######################################

In [17]:
#下面要開始做inception modle

In [18]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)


In [19]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(146, activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [20]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'])

In [31]:
# model.fit(train_data, y, batch_size=64, nb_epoch=50,
#               show_accuracy=True, verbose=1, validation_split=0.2)
hist=model.fit(x=X_train, y=y_train,
              batch_size=32,
              nb_epoch=20,
              validation_data=(X_test,y_test),
              verbose=1)

Train on 4515 samples, validate on 2225 samples
Epoch 1/20
4515/4515 [==============================] - 31s - loss: 3.6787 - acc: 0.1838 - val_loss: 5.1713 - val_acc: 0.1169
Epoch 2/20
4515/4515 [==============================] - 31s - loss: 2.7558 - acc: 0.3517 - val_loss: 3.2860 - val_acc: 0.2912
Epoch 3/20
4515/4515 [==============================] - 31s - loss: 2.4375 - acc: 0.4215 - val_loss: 2.7827 - val_acc: 0.3942
Epoch 4/20
4515/4515 [==============================] - 31s - loss: 2.2128 - acc: 0.4731 - val_loss: 2.5611 - val_acc: 0.4436
Epoch 5/20
4515/4515 [==============================] - 31s - loss: 2.0322 - acc: 0.5110 - val_loss: 2.4313 - val_acc: 0.4629
Epoch 6/20
4515/4515 [==============================] - 31s - loss: 1.9034 - acc: 0.5389 - val_loss: 2.3563 - val_acc: 0.4773
Epoch 7/20
4515/4515 [==============================] - 31s - loss: 1.7911 - acc: 0.5619 - val_loss: 2.2856 - val_acc: 0.5074
Epoch 8/20
4515/4515 [==============================] - 31s - loss: 1.

In [32]:
print(hist.history)

{'val_acc': [0.11685393258594395, 0.29123595506287692, 0.39415730346454664, 0.44359550565816042, 0.46292134835478965, 0.47730337082669977, 0.50741573055138744, 0.5164044943954168, 0.52584269666939643, 0.53168539327182129, 0.52674157324801674, 0.54112359551901223, 0.5573033707999111, 0.56494382043902791, 0.55775280920307291, 0.56134831482105041, 0.55056179796711779, 0.55191011257385936, 0.57168539347273584, 0.57573033726617195], 'acc': [0.18383167220376523, 0.35171650056031062, 0.42148394241417497, 0.47308970099667774, 0.51096345515610242, 0.53887043190028849, 0.56190476190476191, 0.59712070874861578, 0.61018826135765281, 0.63056478405975691, 0.65692137321364441, 0.66400885936429732, 0.6728682170608643, 0.68704318938197229, 0.70011074197780787, 0.71251384275960239, 0.72668881507410954, 0.738648947964475, 0.73399778517377734, 0.75215946845173964], 'val_loss': [5.1713030680109942, 3.2859904569990181, 2.7827015075254975, 2.5611257336648663, 2.4312517365444912, 2.3562885275851473, 2.2855632

In [ ]:
#炫技用 view layers name 

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 172 layers and unfreeze the rest:
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True
